<a href="https://colab.research.google.com/github/minhminh2703/LTKHDL/blob/main/CSC14118_Spark_Coding_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Java and Spark on Hadoop

In [ ]:
# install java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# install spark (change the version number if needed)
!wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
# unzip the spark file to the current folder
!tar xf spark-3.5.1-bin-hadoop3.tgz

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [773 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,356 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,898 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadc

In [ ]:
# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

# Create a SparkSession in Python

In [ ]:
# start pyspark
!pip install findspark
import findspark
findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local")\
          .appName("Introduction to Spark")\
          .config("spark.some.config.option", "some-value")\
          .getOrCreate()

# Example 1: WordCount with Spark DataFrames and Spark RDDs

In [ ]:
# Load the data
!git clone https://github.com/nnthaofit/CSC14118.git

Cloning into 'CSC14118'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (17/17), 818.44 KiB | 4.20 MiB/s, done.
Resolving deltas: 100% (1/1), done.


### Spark DataFrame-based WordCount

In [ ]:
linesDF = spark.read.text("CSC14118/ppap.txt")
linesDF.show(linesDF.count(),truncate = False)

from pyspark.sql import functions as f
wordsDF = linesDF.withColumn("word", f.explode(f.split(f.col("value"), " ")))\
    .groupBy("word")\
    .count()\
    .sort("count", ascending = False)
wordsDF.show()

+----------------------------+
|value                       |
+----------------------------+
|ppap                        |
|i have a pen                |
|i have an apple             |
|ah apple pen                |
|i have a pen                |
|i have a pineapple          |
|ah pineapple pen            |
|ppap pen pineapple apple pen|
+----------------------------+

+---------+-----+
|     word|count|
+---------+-----+
|      pen|    6|
|     have|    4|
|        i|    4|
|    apple|    3|
|pineapple|    3|
|        a|    3|
|     ppap|    2|
|       ah|    2|
|       an|    1|
+---------+-----+



###RDD-based WordCount

In [ ]:
linesRdd = spark.sparkContext.textFile("CSC14118/ppap.txt")
wordsRdd = linesRdd.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b)\
    .sortBy(lambda pair:-1*pair[1])
wordsRdd.collect()

[('pen', 6),
 ('i', 4),
 ('have', 4),
 ('a', 3),
 ('apple', 3),
 ('pineapple', 3),
 ('ppap', 2),
 ('ah', 2),
 ('an', 1)]

# Exercise 1: Data query with Spark DataFrame

###0. Load the data file: movies.json

In [ ]:
# clone the example data files from GitHub to Drive
!git clone https://github.com/nnthaofit/CSC14118.git

In [ ]:
# read the data file and show the first 10 rows
df = spark.read.json("CSC14118/movies.json")
df.show(10, truncate=False)

+-------------+--------------------+-------------------------------------------+----+
|cast         |genres              |title                                      |year|
+-------------+--------------------+-------------------------------------------+----+
|[]           |[]                  |After Dark in Central Park                 |1900|
|[]           |[]                  |Boarding School Girls' Pajama Parade       |1900|
|[]           |[]                  |Buffalo Bill's Wild West Parad             |1900|
|[]           |[]                  |Caught                                     |1900|
|[]           |[]                  |Clowns Spinning Hats                       |1900|
|[]           |[Short, Documentary]|Capture of Boer Battery by British         |1900|
|[]           |[]                  |The Enchanted Drawing                      |1900|
|[Paul Boyton]|[]                  |Feeding Sea Lions                          |1900|
|[]           |[Comedy]            |How to Make a Fat 

### 1a. Show the schema of DataFrame that stores the movies dataset.

### 1b. Show the number of distinct films in the dataset

### 2. Count the number of movies released during the years 2012 and 2015 (included)

### 3. Show the year in which the number of movies released is highest. One highest year is enough

### 4. Show the list of movies such that for each film, the number of actors/actresses is at least five, and the number of genres it belongs to is at most two genres.

### 5. Show the **movies** whose names are longest

### 6. Show the movies whose name contains the word “fighting” (case-insensitive).

### 7. Show the list of distinct genres appearing in the dataset

### 8. List all movies in which the actor Harrison Ford has participated.

### 9. List all movies in which the actors/actresses whose names include the word “Lewis“ (case-insensitive) have participated.

### 10. Show top five actors/actresses that have participated in most movies.